In [ ]:
# 강의 url : https://youtube.com/playlist?list=PL9mhQYIlKEhdrYpsGk8X4qj3tQUuaDhrl
# 참조 논문 : Deep Speech to, Listen attend and spell
# 딥러닝 기초 리뷰 : FC Layer, CNN, RNN, LSTM, Attention
# Audio Classfication & Tagging : 데이터 파이프라인의 이해
# CTC : 논문의 이해, CTC Loss 쓰는 법, Deep Speech2 구현
# LAS : Extra 모델 아키텍쳐 이해

In [ ]:
# Spectrogram 을 input 으로 받고, invariant convolution 을 거친 다음에, RNN (Recurrent Neural Network) 을 거치고, Fully Connected 를 거친다.
# 마지막으로 나온 CTC(Connectionist Temporal Classification) 를 Decode 하여 결과값을 얻어냄.
# IO Module, Network Architecture, Loss 함수만 잘 설정하면 좋은 모델 만들 수 있음.

In [ ]:
# Connectivity Patterns + Nonlinearity modules

# Connectivity Patterns
# - Fully-Connected
# - Convolutional
# - Dilated
# - Recurrent
# - Skip / Residual
# - Random

# Nonlinearity modules
# - ReLU
# - Sigmoid
# - Tanh
# - GRU
# - LSTM

# Loss
# - Cross Entropy
# - Adversarial
# - Variational
# - Maximum Likelihood
# - L1 and L2

# Optimizer
# - SGD
# - Momentum
# - RMSProp
# - Adagrad
# - Adam
# - Second Order

# Hyper Parameters
# - Learning rate
# - Weight decay
# - Layer size
# - Batch size
# - Dropout rate
# - Weight initialization
# - Data augmentation
# - Gradient clipping
# - Momentum

# 13분경 이어서 학습